# Convolution

And how to abuse notation in Julia

In [1]:
using DSP

In [2]:
include("rxsignal_withchannelandfreqoff.jl");

In [3]:
include("pss2.jl");

In [4]:
size(rxs3) # 616447×1 Matrix{ComplexF64}

(616447, 1)

In [6]:
using FFTW

In [7]:
Ŝᵣₓ³ = rxs3 # RX Received Signal 3
PSS₂ = pss_2;
PSS₂ᵀ = transpose.(PSS₂); # Hessian Transpose ?
Fᴵ = ifft # Inverse Fourier Transform
# Convolution infix function
function ⊗(a, b) 
    return conv(a,b)
end

⊗ (generic function with 1 method)

In [8]:
size(PSS₂) # 2048-element Vector{Complex{Int64}}

(2048,)

In [9]:
c = Ŝᵣₓ³ ⊗ PSS₂; size( c ) # 618494×1 Matrix{ComplexF64} which is size(rxs3) + size(pss_2) - 1

(618494, 1)

In [10]:
𝑆ₚₛₛ² = Fᴵ(PSS₂ᵀ); # S Slanted (fourier transform) in time domain . see SSP - Spectral Descriptions

In [11]:
c1 = Ŝᵣₓ³ ⊗ 𝑆ₚₛₛ²; size( c1 ) # 618494×1 Matrix{ComplexF64} which is size(rxs3) + size(pss_2) - 1

(618494, 1)

In [12]:
using LinearAlgebra

In [16]:
𝑆ₚₛₛ² ./= norm(𝑆ₚₛₛ²);

In [17]:
c3 = Ŝᵣₓ³ ⊗ 𝑆ₚₛₛ²; size( c3 ) # 618494×1 Matrix{ComplexF64} which is size(rxs3) + size(pss_2) - 1

(618494, 1)

##### The purpose of this concatenation might be related to handling circular convolution or other signal processing considerations.
end is 2048 in this case, concat math representation would be [ A B }

In [18]:
𝑆ₚₛₛ² = vcat(𝑆ₚₛₛ²[(end-143):end], 𝑆ₚₛₛ²); size(𝑆ₚₛₛ²) # end is 2048 in this case, concat math 

(2192,)

In [19]:
c4 = Ŝᵣₓ³ ⊗ 𝑆ₚₛₛ²; size( c4 ) # 618494×1 Matrix{ComplexF64} which is size(rxs3) + size(pss_2) - 1

(618638, 1)

The equivalent Julia code for `m2_chan = 10*log(abs(conv(rxs3,conj(fliplr(pss2_t)))));` would be as follows:

```julia
m2_chan = 10 * log10(abs(conv(rxs3, conj(reverse(pss2_t)))));
```

Here's a breakdown of the Julia code:

1. `reverse(pss2_t)`: This function reverses the order of elements in the vector `pss2_t`.

2. `conj(reverse(pss2_t))`: This computes the complex conjugate of each element in the reversed vector.

3. `conv(rxs3, conj(reverse(pss2_t)))`: This computes the convolution of the vectors `rxs3` and `conj(reverse(pss2_t))`.

4. `abs(conv(rxs3, conj(reverse(pss2_t))))`: This takes the absolute values of the elements in the result of the convolution.

5. `10 * log10(...)`: This scales the result by a factor of 10 and computes the logarithm base 10 of each element.

The final result is stored in the variable `m2_chan`, representing the logarithmic power envelope of the convolution between the received signal `rxs3` and the complex conjugate of the time-reversed PSS signal in decibels (dB).

The mathematical symbols for reverse and conjugate operations are not standardized across all branches of mathematics, and different notations may be used in different contexts. However, some commonly used notations are:

1. **Reverse Operation:**
$$
\begin{cases*}
   - \text { In some contexts, the reverse (or reverse order) operation is denoted by a bar or overline.}
     \\
     \qquad \text { For example, if } A \text{ is a sequence or vector, the reverse of } A \text{ might be denoted as } \overline{A} \text{  or } A^*.
     \\
   - \text { In signal processing, you might also see the notation }
     \\
     \qquad A[n] \text { for the reverse of } A \text {, where n is the reversal operator. }
\end{cases*}
$$

3. **Conjugate Operation:**
$$
\begin{cases*}
   - \text { The conjugate of a complex number } z \text {  is often denoted by a bar or overline. }
     \\
     \qquad \text { For example, if } z = a + bi \text{ , then } \bar{z} = a - bi.
     \\
   - \text { In linear algebra, the conjugate transpose of a matrix }
        A \text { is often denoted by } A^* \text {  or } A^H.
\end{cases*}
$$

If you are working within a specific mathematical context or field, it's always a good practice to refer to the conventions and notations used within that field.

In [20]:
S̅ₚₛₛ² = reverse(𝑆ₚₛₛ²); # reverse
S̅ₚₛₛ²ᴴ = conj(S̅ₚₛₛ²); # conjugate 
argmax = findmax

findmax (generic function with 8 methods)

In [21]:
# Perform convolution
# m2_chan = 10 * log10.(abs2.(conv(rxs3, conj(reverse(pss2_t))))); ; size( m2_chan )
# m̂₂ = 10 * log.(abs.( Ŝᵣₓ³ ⊗ S̅ₚₛₛ²ᴴ )); ; size( m̂₂ )
m̂₂ = 10 * log10.(abs.( Ŝᵣₓ³ ⊗ S̅ₚₛₛ²ᴴ )); ; size( m̂₂ )

(618638, 1)

In [22]:
# Find maximum value and its index
Ĉᵩ², N̂ᵩ² = argmax(m̂₂) #; m2_chan[NF2_chan] or direct m2_chan[(6628)...] 

(50.649042476081405, CartesianIndex(6628, 1))

In [23]:
 6628 - (2048 + 144) + 1

4437

In [24]:
using Plots

# Plot the result
m2_chan_plot = plot(m̂₂, xlabel="Sample", ylabel="Power (dB)", title="Convolution Result", ylim=(-20, 60))
savefig(m2_chan_plot,"images/m2_chan_plot.png");

<img src="images/m2_chan_plot.png" weight='' height='' > </img>

In [25]:
y_1 = m̂₂[N̂ᵩ²] 
x_1 = getindex(N̂ᵩ², 1)

6628

In [26]:
m2_chan_zoom_plot = xlims!(x_1 - 1000, x_1 + 1000)
m2_chan_zoom_plot = ylims!(y_1 - 30, y_1 + 10)
#scatter!(x_1, y_1, color = "green", label = "", markersize = 10)
savefig(m2_chan_zoom_plot,"images/m2_chan_zoom_plot.png");

<img src="images/m2_chan_zoom_plot.png" weight='' height='' > </img>

# References

- [ ] [Transpose vs. Adjoint?](https://discourse.julialang.org/t/transpose-vs-adjoint/69445)
- [ ] [Convolution of Complex vectors](https://stackoverflow.com/questions/57169773/convolution-of-complex-vectors)
- [ ] [Convolution (conv) with same size output](https://discourse.julialang.org/t/convolution-conv-with-same-size-output/38260/7)
    - I usually expect the convolution of a length-N array and a length-M array to be length N+M-1, but that should be documented.

In [27]:
a = [1, 2, 1, 2] 
b = [1, 2, 3]
@show a; @show b;

a = [1, 2, 1, 2]
b = [1, 2, 3]


In [28]:
aᵪ = @. a + 2.9im # 4-element column vector

4-element Vector{ComplexF64}:
 1.0 + 2.9im
 2.0 + 2.9im
 1.0 + 2.9im
 2.0 + 2.9im

In [29]:
bᵪ = @. b + 4.5im # 3-element column vector

3-element Vector{ComplexF64}:
 1.0 + 4.5im
 2.0 + 4.5im
 3.0 + 4.5im

In [30]:
conv(bᵪ,aᵪ) # 6-element column vector

6-element Vector{ComplexF64}:
              -12.05 + 7.4im
 -22.099999999999998 + 22.199999999999996im
 -31.149999999999995 + 35.39999999999999im
 -29.149999999999995 + 39.89999999999999im
               -19.1 + 28.0im
  -7.050000000000001 + 17.699999999999996im

In [31]:
conv(transpose(bᵪ),aᵪ) # 4x3 Matrix

4×3 Matrix{ComplexF64}:
 -12.05+7.4im   -11.05+10.3im  -10.05+13.2im
 -11.05+11.9im   -9.05+14.8im   -7.05+17.7im
 -12.05+7.4im   -11.05+10.3im  -10.05+13.2im
 -11.05+11.9im   -9.05+14.8im   -7.05+17.7im